In [1]:
byte_string = b'0030630230001000'
decoded_string = byte_string.decode('utf-8')
print(decoded_string)

0030630230001000


In [2]:
import pandas as pd

# Assuming df is your DataFrame with columns 'Instrument', 'Price', 'Time', 'Volume', 'Expiration'
# Replace this with your actual DataFrame
data = {
    'Instrument': ['ESH04', 'ESH04', 'ESH04', 'ESH04', 'ESH04'],
    'Price': [1062.75, 1064.00, 1066.50, 1066.00, 1065.50],
    'Time': ['2003-12-11 08:50:46', '2003-12-11 09:21:45', '2003-12-11 09:51:40', '2003-12-11 10:54:14', '2003-12-11 12:17:37'],
    'Volume': [3, 5, 10, 20, 10],
    'Expiration': ['2004-03-19', '2004-03-19', '2004-03-19', '2004-03-19', '2004-03-19']
}

df = pd.DataFrame(data)
df['Time'] = pd.to_datetime(df['Time'])
df['Expiration'] = pd.to_datetime(df['Expiration'])

# Sort the DataFrame by time
df = df.sort_values(by='Time')

# Create a column 'ContractID' to identify each contract
df['ContractID'] = (df['Instrument'] + '_' + df['Expiration'].dt.strftime('%Y%m%d')).astype('category').cat.codes

# Create a column 'AdjustedPrice' to store the adjusted prices
df['AdjustedPrice'] = df['Price']

# Apply roll adjustments
for contract_id in df['ContractID'].unique():
    contract_mask = df['ContractID'] == contract_id
    roll_index = contract_mask.idxmax()  # Assume rolls happen at the first occurrence in time
    next_contract_index = df[contract_mask].index.min()

    if not pd.isnull(next_contract_index):
        # Adjust prices for the roll
        df.loc[roll_index:next_contract_index, 'AdjustedPrice'] -= df.loc[next_contract_index, 'Price'] - df.loc[roll_index, 'Price']

# Drop unnecessary columns
df = df.drop(['ContractID', 'Expiration'], axis=1)

# Print the resulting DataFrame
print(df)

  Instrument    Price                Time  Volume  AdjustedPrice
0      ESH04  1062.75 2003-12-11 08:50:46       3        1062.75
1      ESH04  1064.00 2003-12-11 09:21:45       5        1064.00
2      ESH04  1066.50 2003-12-11 09:51:40      10        1066.50
3      ESH04  1066.00 2003-12-11 10:54:14      20        1066.00
4      ESH04  1065.50 2003-12-11 12:17:37      10        1065.50
